In [0]:
from pyspark.sql import SparkSession,functions as func

In [0]:
spark=SparkSession.builder.appName("ALS Recommendation").getOrCreate()

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,LongType,StringType,FloatType,DoubleType

In [0]:
schema1=StructType([StructField("user_id",IntegerType(),True),StructField("movie_id",IntegerType(),True),StructField("rating",IntegerType(),True),StructField("timestamp",LongType(),True)])

In [0]:
df=spark.read.option("sep","\t").schema(schema1).csv("dbfs:/FileStore/tables/u.data")
df.show(5)

+-------+--------+------+---------+
user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
 196| 242| 3|881250949|
 186| 302| 3|891717742|
 22| 377| 1|878887116|
 244| 51| 2|880606923|
 166| 346| 1|886397596|
+-------+--------+------+---------+
only showing top 5 rows

In [0]:
schema1=StructType([StructField("movieId",IntegerType(),True),StructField("title",StringType(),True),StructField("genres",StringType(),True)])

In [0]:
movies=spark.read.schema(schema1).option("header","true").csv("dbfs:/FileStore/tables/movies.csv")
movies.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [0]:
movies.createOrReplaceTempView("movie")
df.createOrReplaceTempView("rating")
spark.sql("select r.*,m.title,m.genres from rating r inner join movie m on r.movie_id==m.movieId").show(5)

+-------+--------+------+---------+----------------+--------------------+
user_id|movie_id|rating|timestamp| title| genres|
+-------+--------+------+---------+----------------+--------------------+
 593| 1| 3|875659150|Toy Story (1995)|Adventure|Animati...|
 800| 1| 4|887646283|Toy Story (1995)|Adventure|Animati...|
 786| 1| 4|882841828|Toy Story (1995)|Adventure|Animati...|
 747| 1| 5|888639138|Toy Story (1995)|Adventure|Animati...|
 895| 1| 4|879437950|Toy Story (1995)|Adventure|Animati...|
+-------+--------+------+---------+----------------+--------------------+
only showing top 5 rows

In [0]:
new_df=df.select("user_id","movie_id","rating")
new_df.show(5)

+-------+--------+------+
user_id|movie_id|rating|
+-------+--------+------+
 196| 242| 3|
 186| 302| 3|
 22| 377| 1|
 244| 51| 2|
 166| 346| 1|
+-------+--------+------+
only showing top 5 rows

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
train_data,test_data=new_df.randomSplit([0.7,0.3],seed=1234)

In [0]:
als=ALS(userCol="user_id",itemCol="movie_id",ratingCol="rating",nonnegative=True,coldStartStrategy="drop").setMaxIter(5).setRegParam(0.01)

In [0]:
model=als.fit(train_data)

In [0]:
test_predictions=model.transform(test_data)
test_predictions.show(10)

+-------+--------+------+----------+
user_id|movie_id|rating|prediction|
+-------+--------+------+----------+
 148| 70| 5| 3.3717582|
 148| 71| 5| 4.428236|
 148| 89| 5| 4.0484786|
 148| 114| 5| 6.087761|
 148| 116| 5| 3.2340446|
 148| 127| 1| 3.4954617|
 148| 133| 5| 2.760665|
 148| 135| 5| 4.0656614|
 148| 168| 5| 5.714344|
 148| 169| 5| 5.2635846|
+-------+--------+------+----------+
only showing top 10 rows

In [0]:
evaluator=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")

In [0]:
RMSE=evaluator.evaluate(test_predictions)
print(RMSE)

1.0327622309754132

In [0]:
# Generate top 10 movie recommendations for each user.
nrecommendations=model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+-------+--------------------+
user_id| recommendations|
+-------+--------------------+
 1|[[1643, 5.76429],...|
 3|[[1643, 6.7678995...|
 6|[[1643, 7.1557565...|
 12|[[1512, 9.139532]...|
 13|[[1463, 6.34708],...|
 16|[[1192, 8.608451]...|
 20|[[146, 11.883984]...|
 22|[[1192, 9.977172]...|
 26|[[1192, 5.6740723...|
 27|[[1643, 7.432513]...|
+-------+--------------------+

In [0]:
# Generate top 10 movie recommendations for each user after exploding recommendations column.
nrecommendations=nrecommendations.withColumn("rec_exp",func.explode("recommendations")).\
                 select('user_id',func.col("rec_exp.movie_id"),func.col("rec_exp.rating"))
nrecommendations.limit(10).show()

+-------+--------+---------+
user_id|movie_id| rating|
+-------+--------+---------+
 1| 1643| 5.76429|
 1| 1449|5.7395797|
 1| 745| 5.634434|
 1| 1463|5.4635696|
 1| 1558|5.3780684|
 1| 1192|5.3707514|
 1| 1169| 5.27847|
 1| 390|5.1752806|
 1| 1126| 5.133106|
 1| 953|5.1252027|
+-------+--------+---------+

In [0]:
nrecommendations.createOrReplaceTempView("recommend")

In [0]:
# Generate top 10 movie recommendations for each user along with title & genres.
spark.sql("select re.*,m.title,m.genres from recommend re inner join movie m on re.movie_id==m.movieId").show(10)

+-------+--------+---------+--------------------+--------------------+
user_id|movie_id| rating| title| genres|
+-------+--------+---------+--------------------+--------------------+
 1| 1643| 5.76429|Mrs. Brown (a.k.a...| Drama|Romance|
 1| 1449|5.7395797|Waiting for Guffm...| Comedy|
 1| 745| 5.634434|Wallace & Gromit:...|Animation|Childre...|
 1| 1463|5.4635696|That Old Feeling ...| Comedy|Romance|
 1| 1558|5.3780684|Sudden Manhattan ...| Comedy|
 1| 1192|5.3707514|Paris Is Burning ...| Documentary|
 1| 1169| 5.27847|American Dream (1...| Documentary|
 1| 390|5.1752806|Faster Pussycat! ...| Action|Crime|Drama|
 1| 1126| 5.133106|Drop Dead Fred (1...| Comedy|Fantasy|
 1| 953|5.1252027|It's a Wonderful ...|Children|Drama|Fa...|
+-------+--------+---------+--------------------+--------------------+
only showing top 10 rows

In [0]:
# Generate top 10 movie recommendations for user_id 31 along with title & genres.
recommend_df=spark.sql("select re.*,m.title,m.genres from recommend re inner join movie m on re.movie_id==m.movieId")
recommend_df.filter(recommend_df.user_id==31).sort('rating',ascending=False).limit(10).show()

+-------+--------+---------+--------------------+--------------------+
user_id|movie_id| rating| title| genres|
+-------+--------+---------+--------------------+--------------------+
 31| 1077| 7.982669| Sleeper (1973)| Comedy|Sci-Fi|
 31| 865|7.9143124| Small Faces (1996)| Drama|
 31| 793|7.6695533|My Life and Times...| Drama|
 31| 1240|7.3168077|Terminator, The (...|Action|Sci-Fi|Thr...|
 31| 592|6.9683056| Batman (1989)|Action|Crime|Thri...|
 31| 1036|6.9081516| Die Hard (1988)|Action|Crime|Thri...|
 31| 867|6.8474717| Carpool (1996)| Comedy|Crime|
 31| 1456|6.7370105| Pest, The (1997)| Comedy|
 31| 1154|6.7094603| T-Men (1947)| Film-Noir|
+-------+--------+---------+--------------------+--------------------+

In [0]:
# Generate top 10 user recommendations for each movie.
movieRecs=model.recommendForAllItems(10)
movieRecs.limit(10).show()

+--------+--------------------+
movie_id| recommendations|
+--------+--------------------+
 12|[[261, 6.3384395]...|
 22|[[174, 6.2012086]...|
 26|[[811, 6.140478],...|
 27|[[471, 8.411355],...|
 28|[[732, 5.7535906]...|
 31|[[810, 5.621173],...|
 34|[[202, 13.173415]...|
 44|[[335, 6.2385893]...|
 47|[[153, 10.746742]...|
 53|[[78, 9.722074], ...|
+--------+--------------------+

In [0]:
# Generate top 10 user recommendations for each movie after exploding recommendations column.
movieRecs=movieRecs.withColumn("rec_exp",func.explode("recommendations")).\
                 select('movie_id',func.col("rec_exp.user_id"),func.col("rec_exp.rating"))
movieRecs.limit(10).show()

+--------+-------+---------+
movie_id|user_id| rating|
+--------+-------+---------+
 12| 261|6.3384395|
 12| 353|6.1613407|
 12| 355|5.9877567|
 12| 697| 5.768664|
 12| 4| 5.749005|
 12| 240| 5.696423|
 12| 811|5.6919355|
 12| 550| 5.585383|
 12| 799|5.5616455|
 12| 332| 5.552833|
+--------+-------+---------+

In [0]:
# Generate top 10 user recommendations for each movie along with title & genres.
movieRecs.createOrReplaceTempView("movieRec")
spark.sql("select mr.*,m.title,m.genres from movieRec mr inner join movie m on mr.movie_id=m.movieId").show(10)

+--------+-------+---------+--------------------+-------------+
movie_id|user_id| rating| title| genres|
+--------+-------+---------+--------------------+-------------+
 12| 261|6.3384395|Dracula: Dead and...|Comedy|Horror|
 12| 353|6.1613407|Dracula: Dead and...|Comedy|Horror|
 12| 355|5.9877567|Dracula: Dead and...|Comedy|Horror|
 12| 697| 5.768664|Dracula: Dead and...|Comedy|Horror|
 12| 4| 5.749005|Dracula: Dead and...|Comedy|Horror|
 12| 240| 5.696423|Dracula: Dead and...|Comedy|Horror|
 12| 811|5.6919355|Dracula: Dead and...|Comedy|Horror|
 12| 550| 5.585383|Dracula: Dead and...|Comedy|Horror|
 12| 799|5.5616455|Dracula: Dead and...|Comedy|Horror|
 12| 332| 5.552833|Dracula: Dead and...|Comedy|Horror|
+--------+-------+---------+--------------------+-------------+
only showing top 10 rows

In [0]:
# Generate top 10 user recommendations for each movie along with title & genres.
movieRec_df=spark.sql("select mr.*,m.title,m.genres from movieRec mr inner join movie m on mr.movie_id=m.movieId")
movieRec_df.filter(movieRec_df.movie_id==31).sort('rating',ascending=False).limit(10).show()

+--------+-------+---------+--------------------+------+
movie_id|user_id| rating| title|genres|
+--------+-------+---------+--------------------+------+
 31| 810| 5.621173|Dangerous Minds (...| Drama|
 31| 443| 5.509704|Dangerous Minds (...| Drama|
 31| 939| 5.462859|Dangerous Minds (...| Drama|
 31| 850| 5.438737|Dangerous Minds (...| Drama|
 31| 808| 5.427509|Dangerous Minds (...| Drama|
 31| 4| 5.217102|Dangerous Minds (...| Drama|
 31| 351| 5.191621|Dangerous Minds (...| Drama|
 31| 403|5.1765795|Dangerous Minds (...| Drama|
 31| 357| 5.151716|Dangerous Minds (...| Drama|
 31| 317|5.0986795|Dangerous Minds (...| Drama|
+--------+-------+---------+--------------------+------+

In [0]:
# Generate top 10 movie recommendations for a specified set of users
users=new_df.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs=model.recommendForUserSubset(users,10)
userSubsetRecs.show()

+-------+--------------------+
user_id| recommendations|
+-------+--------------------+
 22|[[1192, 9.977172]...|
 196|[[1598, 9.443933]...|
 186|[[909, 7.487275],...|
+-------+--------------------+

In [0]:
# Generate top 10 movie recommendations for a specified set of users after exploding recommendations column.
userSubsetRecs=userSubsetRecs.withColumn("rec_exp",func.explode("recommendations")).\
                 select('user_id',func.col("rec_exp.movie_id"),func.col("rec_exp.rating"))
userSubsetRecs.show()

+-------+--------+---------+
user_id|movie_id| rating|
+-------+--------+---------+
 22| 1192| 9.977172|
 22| 695|6.4481316|
 22| 915|6.2445583|
 22| 113| 6.127125|
 22| 1184|5.9937634|
 22| 1598|5.9208765|
 22| 344| 5.892812|
 22| 1262| 5.795911|
 22| 1282|5.7803855|
 22| 854|5.6933117|
 196| 1598| 9.443933|
 196| 634| 8.646293|
 196| 695| 8.57574|
 196| 1069| 8.439869|
 196| 1174|8.1518135|
 196| 1512| 8.105495|
 196| 650|7.7720456|
 196| 536| 7.749665|
 196| 1114|7.5785065|
 196| 1172| 7.543298|
+-------+--------+---------+
only showing top 20 rows

In [0]:
# Generate top 10 movie recommendations for a specified set of users along with title & genres.
userSubsetRecs.createOrReplaceTempView("Usersubset")
spark.sql("select u.*,m.title,m.genres from Usersubset u inner join movie m on u.movie_id=m.movieId").show(10)

+-------+--------+---------+--------------------+--------------------+
user_id|movie_id| rating| title| genres|
+-------+--------+---------+--------------------+--------------------+
 22| 1192| 9.977172|Paris Is Burning ...| Documentary|
 22| 695|6.4481316| True Crime (1996)| Mystery|Thriller|
 22| 915|6.2445583| Sabrina (1954)| Comedy|Romance|
 22| 113| 6.127125|Before and After ...| Drama|Mystery|
 22| 1184|5.9937634| Mediterraneo (1991)| Comedy|Drama|
 22| 1598|5.9208765|Desperate Measure...|Crime|Drama|Thriller|
 22| 344| 5.892812|Ace Ventura: Pet ...| Comedy|
 22| 1262| 5.795911|Great Escape, The...|Action|Adventure|...|
 22| 1282|5.7803855| Fantasia (1940)|Animation|Childre...|
 22| 854|5.6933117|Ballad of Narayam...| Drama|
+-------+--------+---------+--------------------+--------------------+
only showing top 10 rows

In [0]:
# Generate top 10 user recommendations for a specified set of movies.
movies=new_df.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs=model.recommendForItemSubset(movies,10)
movieSubSetRecs.show()

+--------+--------------------+
movie_id| recommendations|
+--------+--------------------+
 242|[[174, 6.303267],...|
 302|[[261, 6.1863346]...|
 377|[[166, 8.941542],...|
+--------+--------------------+

In [0]:
# Generate top 10 user recommendations for a specified set of movies after exploding recommendations column.
movieSubSetRecs=movieSubSetRecs.withColumn("rec_exp",func.explode("recommendations")).\
                 select('movie_id',func.col("rec_exp.user_id"),func.col("rec_exp.rating"))
movieSubSetRecs.show()

+--------+-------+---------+
movie_id|user_id| rating|
+--------+-------+---------+
 242| 174| 6.303267|
 242| 220|6.1601334|
 242| 78| 5.984634|
 242| 71| 5.982554|
 242| 146| 5.888129|
 242| 462| 5.881028|
 242| 261|5.8485723|
 242| 820| 5.748246|
 242| 335|5.6423545|
 242| 353|5.6399474|
 302| 261|6.1863346|
 302| 434|6.1821203|
 302| 550| 6.085868|
 302| 137|5.9188523|
 302| 355| 5.882231|
 302| 324|5.7805114|
 302| 811|5.7574706|
 302| 217| 5.73351|
 302| 4|5.7012696|
 302| 153|5.6396966|
+--------+-------+---------+
only showing top 20 rows

In [0]:
# Generate top 10 user recommendations for a specified set of movies along with title & genres.
movieSubSetRecs.createOrReplaceTempView("movieSubset")
spark.sql("select s.*,m.title,m.genres from movieSubset s inner join movie m on s.movie_id=m.movieId").show(10)

+--------+-------+---------+--------------------+-------------+
movie_id|user_id| rating| title| genres|
+--------+-------+---------+--------------------+-------------+
 242| 174| 6.303267|Farinelli: il cas...|Drama|Musical|
 242| 220|6.1601334|Farinelli: il cas...|Drama|Musical|
 242| 78| 5.984634|Farinelli: il cas...|Drama|Musical|
 242| 71| 5.982554|Farinelli: il cas...|Drama|Musical|
 242| 146| 5.888129|Farinelli: il cas...|Drama|Musical|
 242| 462| 5.881028|Farinelli: il cas...|Drama|Musical|
 242| 261|5.8485723|Farinelli: il cas...|Drama|Musical|
 242| 820| 5.748246|Farinelli: il cas...|Drama|Musical|
 242| 335|5.6423545|Farinelli: il cas...|Drama|Musical|
 242| 353|5.6399474|Farinelli: il cas...|Drama|Musical|
+--------+-------+---------+--------------------+-------------+
only showing top 10 rows

In [0]:
user_id=148
user_schema=StructType([StructField("user_id",IntegerType(),True)])
users=spark.createDataFrame([[user_id,]],user_schema)

In [0]:
recommendations=model.recommendForUserSubset(users,10)
recommendations.show()

+-------+--------------------+
user_id| recommendations|
+-------+--------------------+
 148|[[1512, 11.486127...|
+-------+--------------------+

In [0]:
display(recommendations)

user_id,recommendations
148,"List(List(1512, 11.486127), List(634, 10.046826), List(650, 9.226193), List(1598, 8.983273), List(1114, 8.9826565), List(1069, 8.96747), List(1036, 8.640676), List(536, 8.638755), List(835, 8.557542), List(1615, 8.47964))"


In [0]:
!pip install mlflow

Requirement already satisfied: mlflow in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (1.12.1)
Requirement already satisfied: click>=7.0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mlflow) (7.0)
Requirement already satisfied: querystring-parser in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: databricks-cli>=0.8.7 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mlflow) (0.14.0)
Requirement already satisfied: gitpython>=2.1.0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mlflow) (3.1.0)
Collecting alembic<=1.4.1
 Downloading alembic-1.4.1.tar.gz (1.1 MB)
▎ | 10 kB 11.5 MB/s eta 0:00:01 |▋ | 20 kB 12.1 MB/s eta 0:00:01 |█ | 30 kB 14.0 MB/s eta 0:00:01 |█▏ | 40 kB 9.2 MB/s eta 0:00:01 |█▌ | 51 kB 8.4 MB/s eta 0:00:01 |█▉ | 61 kB 5.9 MB/s eta 0:00:01 |██▏ | 71 kB 6.2 MB/s eta 0:00:01 |██▍ | 81 kB 6.0 MB/s eta 0:00:01 |██▊ | 92 kB 6.0 MB/s eta 0:00:01 |███ | 102 kB 6.3 MB/s eta 0:00:01 |███▎ | 112 kB 6.3 MB/s eta 0:00:01 |███▋ | 122 kB 6.3 MB/s eta 0:00:01 |████ | 133 kB 6.3 MB/s eta 0:00:01 |████▎ | 143 kB 6.3 MB/s eta 0:00:01 |████▌ | 153 kB 6.3 MB/s eta 0:00:01 |████▉ | 163 kB 6.3 MB/s eta 0:00:01 |█████▏ | 174 kB 6.3 MB/s eta 0:00:01 |█████▍ | 184 kB 6.3 MB/s eta 0:00:01 |█████▊ | 194 kB 6.3 MB/s eta 0:00:01 |██████ | 204 kB 6.3 MB/s eta 0:00:01 |██████▍ | 215 kB 6.3 MB/s eta 0:00:01 |██████▋ | 225 kB 6.3 MB/s eta 0:00:01 |███████ | 235 kB 6.3 MB/s eta 0:00:01 |███████▎ | 245 kB 6.3 MB/s eta 0:00:01 |███████▌ | 256 kB 6.3 MB/s eta 0:00:01 |███████▉ | 266 kB 6.3 MB/s eta 0:00:01 |████████▏ | 276 kB 6.3 MB/s eta 0:00:01 |████████▌ | 286 kB 6.3 MB/s eta 0:00:01 |████████▊ | 296 kB 6.3 MB/s eta 0:00:01 |█████████ | 307 kB 6.3 MB/s eta 0:00:01 |█████████▍ | 317 kB 6.3 MB/s eta 0:00:01 |█████████▋ | 327 kB 6.3 MB/s eta 0:00:01 |██████████ | 337 kB 6.3 MB/s eta 0:00:01 |██████████▎ | 348 kB 6.3 MB/s eta 0:00:01 |██████████▋ | 358 kB 6.3 MB/s eta 0:00:01 |██████████▉ | 368 kB 6.3 MB/s eta 0:00:01 |███████████▏ | 378 kB 6.3 MB/s eta 0:00:01 |███████████▌ | 389 kB 6.3 MB/s eta 0:00:01 |███████████▉ | 399 kB 6.3 MB/s eta 0:00:01 |████████████ | 409 kB 6.3 MB/s eta 0:00:01 |████████████▍ | 419 kB 6.3 MB/s eta 0:00:01 |████████████▊ | 430 kB 6.3 MB/s eta 0:00:01 |█████████████ | 440 kB 6.3 MB/s eta 0:00:01 |█████████████▎ | 450 kB 6.3 MB/s eta 0:00:01 |█████████████▋ | 460 kB 6.3 MB/s eta 0:00:01 |██████████████ | 471 kB 6.3 MB/s eta 0:00:01 |██████████████▏ | 481 kB 6.3 MB/s eta 0:00:01 |██████████████▌ | 491 kB 6.3 MB/s eta 0:00:01 |██████████████▉ | 501 kB 6.3 MB/s eta 0:00:01 |███████████████ | 512 kB 6.3 MB/s eta 0:00:01 |███████████████▍ | 522 kB 6.3 MB/s eta 0:00:01 |███████████████▊ | 532 kB 6.3 MB/s eta 0:00:01 |████████████████ | 542 kB 6.3 MB/s eta 0:00:01 |████████████████▎ | 552 kB 6.3 MB/s eta 0:00:01 |████████████████▋ | 563 kB 6.3 MB/s eta 0:00:01 |█████████████████ | 573 kB 6.3 MB/s eta 0:00:01 |█████████████████▏ | 583 kB 6.3 MB/s eta 0:00:01 |█████████████████▌ | 593 kB 6.3 MB/s eta 0:00:01 |█████████████████▉ | 604 kB 6.3 MB/s eta 0:00:01 |██████████████████▏ | 614 kB 6.3 MB/s eta 0:00:01 |██████████████████▍ | 624 kB 6.3 MB/s eta 0:00:01 |██████████████████▊ | 634 kB 6.3 MB/s eta 0:00:01 |███████████████████ | 645 kB 6.3 MB/s eta 0:00:01 |███████████████████▎ | 655 kB 6.3 MB/s eta 0:00:01 |███████████████████▋ | 665 kB 6.3 MB/s eta 0:00:01 |████████████████████ | 675 kB 6.3 MB/s eta 0:00:01 |████████████████████▎ | 686 kB 6.3 MB/s eta 0:00:01 |████████████████████▌ | 696 kB 6.3 MB/s eta 0:00:01 |████████████████████▉ | 706 kB 6.3 MB/s eta 0:00:01 |█████████████████████▏ | 716 kB 6.3 MB/s eta 0:00:01 |█████████████████████▌ | 727 kB 6.3 MB/s eta 0:00:01 |█████████████████████▊ | 737 kB 6.3 MB/s eta 0:00:01 |██████████████████████ | 747 kB 6.3 MB/s eta 0:00:01 |█████████

In [0]:
import mlflow
with mlflow.start_run():
  als=ALS(userCol="user_id",itemCol="movie_id",ratingCol="rating",nonnegative=True,implicitPrefs=False,coldStartStrategy="drop",maxIter=5,regParam=0.01)
  print(type(als))
  model=als.fit(train_data)
  test_predictions=model.transform(test_data)
  print(test_predictions.show(5))
  evaluator=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
  RMSE=evaluator.evaluate(test_predictions)
  print(RMSE)

<class 'pyspark.ml.recommendation.ALS'>
+-------+--------+------+----------+
user_id|movie_id|rating|prediction|
+-------+--------+------+----------+
 148| 70| 5| 3.3717582|
 148| 71| 5| 4.428236|
 148| 89| 5| 4.0484786|
 148| 114| 5| 6.087761|
 148| 116| 5| 3.2340446|
+-------+--------+------+----------+
only showing top 5 rows

None
1.0327622309754132

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
import mlflow

In [0]:
with mlflow.start_run():
  als=ALS(userCol="user_id",itemCol="movie_id",ratingCol="rating",nonnegative=True,implicitPrefs=False,coldStartStrategy="drop")
  print(type(als))
  param_grid=ParamGridBuilder().addGrid(als.rank,[10]).addGrid(als.regParam,[.01]).addGrid(als.maxIter,[5]).build()
  print(len(param_grid))
  evaluator=RegressionEvaluator(labelCol="rating",predictionCol="prediction",metricName="rmse")
  cv=CrossValidator(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator,numFolds=3)
  print(cv)
  model=cv.fit(train_data)
  best_model=model.bestModel
  print(type(best_model))
  print("Rank:", best_model._java_obj.parent().getRank())
  print("MaxIter:", best_model._java_obj.parent().getMaxIter())
  print("RegParam:", best_model._java_obj.parent().getRegParam())
  test_predictions = best_model.transform(test_data)
  print(test_predictions.show(5))
  RMSE = evaluator.evaluate(test_predictions)
  print(str(RMSE))

<class 'pyspark.ml.recommendation.ALS'>
1
CrossValidator_edfcf156115b
MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
<class 'pyspark.ml.recommendation.ALSModel'>
Rank: 10
MaxIter: 5
RegParam: 0.01
+-------+--------+------+----------+
user_id|movie_id|rating|prediction|
+-------+--------+------+----------+
 148| 70| 5| 3.3717582|
 148| 71| 5| 4.428236|
 148| 89| 5| 4.0484786|
 148| 114| 5| 6.087761|
 148| 116| 5| 3.2340446|
+-------+--------+------+----------+
only showing top 5 rows

None
1.0327622309754132

In [0]:
# Generate top 10 movie recommendations for each user.
nrecommendations1=best_model.recommendForAllUsers(10)
nrecommendations1.limit(10).show()

/databricks/spark/python/pyspark/sql/context.py:120: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 DeprecationWarning)
+-------+--------------------+
user_id| recommendations|
+-------+--------------------+
 1|[[1643, 5.76429],...|
 3|[[1643, 6.7678995...|
 6|[[1643, 7.1557565...|
 12|[[1512, 9.139532]...|
 13|[[1463, 6.34708],...|
 16|[[1192, 8.608451]...|
 20|[[146, 11.883984]...|
 22|[[1192, 9.977172]...|
 26|[[1192, 5.6740723...|
 27|[[1643, 7.432513]...|
+-------+--------------------+

In [0]:
# Generate top 10 movie recommendations for each user after exploding recommendations column.
nrecommendations1=nrecommendations1.withColumn("rec_exp",func.explode("recommendations")).\
                 select('user_id',func.col("rec_exp.movie_id"),func.col("rec_exp.rating"))
nrecommendations1.limit(10).show()

+-------+--------+---------+
user_id|movie_id| rating|
+-------+--------+---------+
 1| 1643| 5.76429|
 1| 1449|5.7395797|
 1| 745| 5.634434|
 1| 1463|5.4635696|
 1| 1558|5.3780684|
 1| 1192|5.3707514|
 1| 1169| 5.27847|
 1| 390|5.1752806|
 1| 1126| 5.133106|
 1| 953|5.1252027|
+-------+--------+---------+

In [0]:
nrecommendations1.createOrReplaceTempView("recommends")

In [0]:
# Generate top 10 movie recommendations for each user along with title & genres.
spark.sql("select re.*,m.title,m.genres from recommends re inner join movie m on re.movie_id==m.movieId").show(10)

+-------+--------+---------+--------------------+--------------------+
user_id|movie_id| rating| title| genres|
+-------+--------+---------+--------------------+--------------------+
 1| 1643| 5.76429|Mrs. Brown (a.k.a...| Drama|Romance|
 1| 1449|5.7395797|Waiting for Guffm...| Comedy|
 1| 745| 5.634434|Wallace & Gromit:...|Animation|Childre...|
 1| 1463|5.4635696|That Old Feeling ...| Comedy|Romance|
 1| 1558|5.3780684|Sudden Manhattan ...| Comedy|
 1| 1192|5.3707514|Paris Is Burning ...| Documentary|
 1| 1169| 5.27847|American Dream (1...| Documentary|
 1| 390|5.1752806|Faster Pussycat! ...| Action|Crime|Drama|
 1| 1126| 5.133106|Drop Dead Fred (1...| Comedy|Fantasy|
 1| 953|5.1252027|It's a Wonderful ...|Children|Drama|Fa...|
+-------+--------+---------+--------------------+--------------------+
only showing top 10 rows

In [0]:
# Generate top 10 movie recommendations for user_id 31 along with title & genres.
recommends_df=spark.sql("select re.*,m.title,m.genres from recommends re inner join movie m on re.movie_id==m.movieId")
recommends_df.filter(recommends_df.user_id==31).sort('rating',ascending=False).limit(10).show()

+-------+--------+---------+--------------------+--------------------+
user_id|movie_id| rating| title| genres|
+-------+--------+---------+--------------------+--------------------+
 31| 1077| 7.982669| Sleeper (1973)| Comedy|Sci-Fi|
 31| 865|7.9143124| Small Faces (1996)| Drama|
 31| 793|7.6695533|My Life and Times...| Drama|
 31| 1240|7.3168077|Terminator, The (...|Action|Sci-Fi|Thr...|
 31| 592|6.9683056| Batman (1989)|Action|Crime|Thri...|
 31| 1036|6.9081516| Die Hard (1988)|Action|Crime|Thri...|
 31| 867|6.8474717| Carpool (1996)| Comedy|Crime|
 31| 1456|6.7370105| Pest, The (1997)| Comedy|
 31| 1154|6.7094603| T-Men (1947)| Film-Noir|
+-------+--------+---------+--------------------+--------------------+

In [0]:
# Generate top 10 user recommendations for each movie.
movieRecs1=best_model.recommendForAllItems(10)
movieRecs1.limit(10).show()

+--------+--------------------+
movie_id| recommendations|
+--------+--------------------+
 12|[[261, 6.3384395]...|
 22|[[174, 6.2012086]...|
 26|[[811, 6.140478],...|
 27|[[471, 8.411355],...|
 28|[[732, 5.7535906]...|
 31|[[810, 5.621173],...|
 34|[[202, 13.173415]...|
 44|[[335, 6.2385893]...|
 47|[[153, 10.746742]...|
 53|[[78, 9.722074], ...|
+--------+--------------------+

In [0]:
# Generate top 10 user recommendations for each movie after exploding recommendations column.
movieRecs1=movieRecs1.withColumn("rec_exp",func.explode("recommendations")).\
                 select('movie_id',func.col("rec_exp.user_id"),func.col("rec_exp.rating"))
movieRecs1.limit(10).show()

+--------+-------+---------+
movie_id|user_id| rating|
+--------+-------+---------+
 12| 261|6.3384395|
 12| 353|6.1613407|
 12| 355|5.9877567|
 12| 697| 5.768664|
 12| 4| 5.749005|
 12| 240| 5.696423|
 12| 811|5.6919355|
 12| 550| 5.585383|
 12| 799|5.5616455|
 12| 332| 5.552833|
+--------+-------+---------+

In [0]:
# Generate top 10 user recommendations for each movie along with title & genres.
movieRecs1.createOrReplaceTempView("movieRecs")
spark.sql("select mr.*,m.title,m.genres from movieRecs mr inner join movie m on mr.movie_id=m.movieId").show(10)

+--------+-------+---------+--------------------+-------------+
movie_id|user_id| rating| title| genres|
+--------+-------+---------+--------------------+-------------+
 12| 261|6.3384395|Dracula: Dead and...|Comedy|Horror|
 12| 353|6.1613407|Dracula: Dead and...|Comedy|Horror|
 12| 355|5.9877567|Dracula: Dead and...|Comedy|Horror|
 12| 697| 5.768664|Dracula: Dead and...|Comedy|Horror|
 12| 4| 5.749005|Dracula: Dead and...|Comedy|Horror|
 12| 240| 5.696423|Dracula: Dead and...|Comedy|Horror|
 12| 811|5.6919355|Dracula: Dead and...|Comedy|Horror|
 12| 550| 5.585383|Dracula: Dead and...|Comedy|Horror|
 12| 799|5.5616455|Dracula: Dead and...|Comedy|Horror|
 12| 332| 5.552833|Dracula: Dead and...|Comedy|Horror|
+--------+-------+---------+--------------------+-------------+
only showing top 10 rows

In [0]:
# Generate top 10 user recommendations for each movie along with title & genres.
movieRecs_df=spark.sql("select mr.*,m.title,m.genres from movieRecs mr inner join movie m on mr.movie_id=m.movieId")
movieRecs_df.filter(movieRecs_df.movie_id==12).sort('rating',ascending=False).limit(10).show()

+--------+-------+---------+--------------------+-------------+
movie_id|user_id| rating| title| genres|
+--------+-------+---------+--------------------+-------------+
 12| 261|6.3384395|Dracula: Dead and...|Comedy|Horror|
 12| 353|6.1613407|Dracula: Dead and...|Comedy|Horror|
 12| 355|5.9877567|Dracula: Dead and...|Comedy|Horror|
 12| 697| 5.768664|Dracula: Dead and...|Comedy|Horror|
 12| 4| 5.749005|Dracula: Dead and...|Comedy|Horror|
 12| 240| 5.696423|Dracula: Dead and...|Comedy|Horror|
 12| 811|5.6919355|Dracula: Dead and...|Comedy|Horror|
 12| 550| 5.585383|Dracula: Dead and...|Comedy|Horror|
 12| 799|5.5616455|Dracula: Dead and...|Comedy|Horror|
 12| 332| 5.552833|Dracula: Dead and...|Comedy|Horror|
+--------+-------+---------+--------------------+-------------+